### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score,\
log_loss,confusion_matrix
import math
import warnings
warnings.filterwarnings('ignore')

### Reading Data

In [2]:
df=pd.read_csv("bank.xls")

In [3]:
df.drop(['ID'],axis=1,inplace=True)

### Downsampling Data

In [4]:
one_class=df[df['Personal Loan']==1]
zero_class=df[df['Personal Loan']==0]

In [5]:
zero_downsample = resample(zero_class,
             replace=True,
             n_samples=len(one_class),
             random_state=42)

In [6]:
data=pd.concat([zero_downsample,one_class])

### Splitting Data

In [7]:
X=data.drop(['Personal Loan'],axis=1).values
Y=data['Personal Loan'].values

In [8]:
X_train,X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
X_train.shape

(768, 12)

In [10]:
sc=StandardScaler()

In [11]:
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

### Cultural Algorithm for Weight Optimisation

In [12]:
class CulturalNN:
    def __init__(self, X_train, y_train,hidden_layer=10, chromosomes=30,
                 belief_space_agents=10, generations=30, mating_parents=10, crossover_rate=0.8,
                 mutation_rate=0.1):
        self.X_train = X_train
        self.y_train = y_train
        self.hidden_layer = hidden_layer
        self.chromosomes = chromosomes
        self.belief_space_agents = belief_space_agents
        self.generations = generations
        self.mating_parents = mating_parents
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.input_layer = X_train.shape[1]
        self.output_layer = 1
        self.genes = self.input_layer*self.hidden_layer + self.hidden_layer + \
            self.hidden_layer*self.output_layer + self.output_layer
        self.pop_size = (self.chromosomes, self.genes)
        self.population = np.random.rand(*self.pop_size)
        self.population = 2*self.population - 1
        self.belief_space = np.random.rand(self.belief_space_agents, self.genes)
        self.belief_space = 2*self.belief_space - 1
        self.best_weights=None

    def sigmoid(self,x):
        return 1/(1 + np.exp(-x))

    def relu(self,x):
        return np.maximum(0, x)

    def forward(self, inputs, weights):
        weight1 = np.reshape(weights[:self.hidden_layer*self.input_layer], (self.hidden_layer, self.input_layer))
        bias1 = np.reshape(weights[self.hidden_layer*self.input_layer:self.hidden_layer*self.input_layer+self.hidden_layer], (self.output_layer,self.hidden_layer))
        weight2 = np.reshape(weights[self.hidden_layer*self.input_layer+self.hidden_layer:-1], (self.output_layer, self.hidden_layer))
        bias2 = np.reshape(weights[-1], (self.output_layer, 1))

        A0 = inputs
        Z1 = A0.dot(weight1.T)+bias1
        A1 = self.relu(Z1)
        Z2 = A1.dot(weight2.T)+bias2
        A2 = self.sigmoid(Z2)
        y_pred = np.round(A2)

        return y_pred

    def fitness_function(self, inputs, population, y_train):
        fitness = []
        for weights in population:
            y_pred = self.forward(inputs, weights)
            acc = accuracy_score(y_train, y_pred)
            fitness.append(acc)

        return fitness

    def acceptance_function(self, belief_space, new_agent, fitness, new_agent_fitness):
        min_fitness_index = np.argmin(fitness)

        if fitness[min_fitness_index] < new_agent_fitness:
            belief_space[min_fitness_index, :] = new_agent

        return belief_space

    def influence_function(self, mating_parents, fitness_belief, fitness_population):
        num_belief_parents = mating_parents//2
        parents_belief_space = self.belief_space[np.random.choice(self.belief_space.shape[0], num_belief_parents, replace=False)]
        sorted_pop = self.population[fitness_population.argsort()]
        num_pop_parents = mating_parents - num_belief_parents
        parents_population = sorted_pop[:num_pop_parents, :]
        parents = np.concatenate((parents_belief_space, parents_population))

        return parents
    
    
    def crossover(self, parents, offspring_size):
        offspring = np.empty(offspring_size)

        crossover_point = np.uint8(offspring_size[1] / 2)

        for k in range(offspring_size[0]):
            parent1_index = k % parents.shape[0]
            parent2_index = (k + 1) % parents.shape[0]

            # Generate a random number between 0 and 1
            r = np.random.rand()

            # If random number is less than crossover rate, perform crossover
            if r < self.crossover_rate:
                offspring[k, 0:crossover_point] = parents[parent1_index, 0:crossover_point]
                offspring[k, crossover_point:] = parents[parent2_index, crossover_point:]
            else:
                offspring[k, :] = parents[parent1_index, :]

        return offspring

    
    def mutation(self, offspring_crossover):
        num_mutations = np.round(self.mutation_rate * self.genes)
        mutation_indices = np.random.randint(0, offspring_crossover.shape[0], int(num_mutations))
        for idx in mutation_indices:
            random_value = np.random.uniform(-1.0, 1.0, 1)
            gene_idx = np.random.randint(0, self.genes, 1)
            offspring_crossover[idx, gene_idx] = random_value

        return offspring_crossover

    def run(self):
        for i in range(self.generations):
            fitness_population = self.fitness_function(self.X_train, self.population, self.y_train)
            fitness_population=np.array(fitness_population)
            
            fitness_belief = self.fitness_function(self.X_train, self.belief_space, self.y_train)
            

            self.belief_space = self.acceptance_function(self.belief_space, self.population[0, :], fitness_belief, fitness_population[0])
            
            parents = self.influence_function(self.mating_parents, fitness_belief, fitness_population)

            offspring_size = (self.chromosomes - parents.shape[0], self.genes)
            
            offspring_crossover = self.crossover(parents,offspring_size)
            
            offspring_mutation = self.mutation(offspring_crossover)
            
            #self.population[0, :] = self.belief_space[np.argmin(fitness_belief), :]
            
            #self.population[1:, :] = offspring_mutation[1:, :]
            
            self.population[0:parents.shape[0],:]=parents
            
            self.population[parents.shape[0]:,:]=offspring_mutation

            #best_fitness = np.max(np.concatenate((fitness_belief, fitness_population)))
            fitness=self.fitness_function(self.X_train,self.belief_space,self.y_train)
            best_match_index=np.where(fitness==np.max(fitness))
            best_fitness=fitness[best_match_index[0][0]]

            print("Generation: {}/{} | Best fitness: {}".format(i+1, self.generations, best_fitness))

        self.best_weights=self.belief_space[np.where(fitness == np.max(fitness))][0]
        
    def predict(self,X_test):
        prediction=self.forward(X_test,self.best_weights)
        return prediction
    
    def reports(self,X_test,y_test):
        prediction=self.forward(X_test,self.best_weights)
        
        print(f"ACCURACY : {accuracy_score(y_test,prediction)}\n\n\
CLASSIFICATION REPORT :\n {classification_report(y_test,prediction)}")


In [13]:
culturalnn=CulturalNN(X_train,y_train)

In [14]:
culturalnn.run()

Generation: 1/30 | Best fitness: 0.7330729166666666
Generation: 2/30 | Best fitness: 0.7330729166666666
Generation: 3/30 | Best fitness: 0.7330729166666666
Generation: 4/30 | Best fitness: 0.7330729166666666
Generation: 5/30 | Best fitness: 0.7330729166666666
Generation: 6/30 | Best fitness: 0.7330729166666666
Generation: 7/30 | Best fitness: 0.7330729166666666
Generation: 8/30 | Best fitness: 0.7330729166666666
Generation: 9/30 | Best fitness: 0.7330729166666666
Generation: 10/30 | Best fitness: 0.7330729166666666
Generation: 11/30 | Best fitness: 0.7330729166666666
Generation: 12/30 | Best fitness: 0.7330729166666666
Generation: 13/30 | Best fitness: 0.7330729166666666
Generation: 14/30 | Best fitness: 0.7330729166666666
Generation: 15/30 | Best fitness: 0.7330729166666666
Generation: 16/30 | Best fitness: 0.7330729166666666
Generation: 17/30 | Best fitness: 0.7330729166666666
Generation: 18/30 | Best fitness: 0.7330729166666666
Generation: 19/30 | Best fitness: 0.7330729166666666
Ge

In [15]:
culturalnn.reports(X_test,y_test)

ACCURACY : 0.734375

CLASSIFICATION REPORT :
               precision    recall  f1-score   support

           0       0.64      0.93      0.76        86
           1       0.91      0.58      0.71       106

    accuracy                           0.73       192
   macro avg       0.78      0.75      0.73       192
weighted avg       0.79      0.73      0.73       192

